In [3]:
import requests
from bs4 import BeautifulSoup
import boto3
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re
import multiprocessing as mp

In [4]:
def request_song_info(song_title, artist_name):
    endpoint = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + BEARER_TOKEN}
    search_url = endpoint + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    for hit in response.json()['response']['hits']:
        hit = hit['result']
        if song_title == hit['title']:
            return hit['url']

    return None

def scrape(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()

    return lyrics

def get_basketball_players():
    players = []
    endpoint = "https://www.basketball-reference.com/players/"
    for letter in range(ord('a'), ord('z')+1):
        reference_page = requests.get(endpoint + chr(letter))
        reference_html = BeautifulSoup(reference_page.text, 'html.parser')
        table = reference_html.find('table')
        if table:
            for row in table.findAll('tr'):
                if row.find('a'):
                    player = row.find('a').text
                    players.append(player)
                    
    return players

def get_artists_or_groups():
    endpoint = "https://en.wikipedia.org/wiki/List_of_hip_hop_musicians"
    page = requests.get(endpoint)
    html = BeautifulSoup(page.text, 'html.parser')
    artists = []
    for li in html.findAll('li'):
        if li.find('a') and li.find('a').get('title'):
            text = li.find('a').get('title')
            artists.append( text )
            
    endpoint = "https://en.wikipedia.org/wiki/List_of_hip_hop_groups"
    page = requests.get(endpoint)
    html = BeautifulSoup(page.text, 'html.parser')
    for li in html.findAll('li'):
        if li.find('a') and li.find('a').get('title'):
            text = li.find('a').get('title')
            artists.append( text )
            
    return artists

def poll(queue):
    with open('queue.txt', 'a', 1) as infile:
        while True:
            message = queue.get()
            print("writing message " + message[0])
            if m == '-1':
                infile.write('killed')
                break
            else:
                infile.write(",".join(m) + "\n")
                infile.flush()

def run(artist):
    shoutouts = []
    results = sp.search(q=artist, limit=50)
    for item in results['tracks']['items']:
        song = item['name']
        if song in songs:
            continue 
        else:
            songs.append(players)
        response = request_song_info(song, artist)
        if not response: continue
        lyrics_list = []
        lyrics = scrape(response).strip().replace('\n', '. ')
#         segments = len(lyrics.encode('utf-8')) // 5000 + 1
#         for s in range(segments):
#             lyrics.append(" ".join(lyrics.split()[:len(lyrics.split())//segments]))
#             lyrics = " ".join(lyrics.split()[len(lyrics.split())//segments:])
            
        try:
            if len(lyrics.encode('utf-8')) > 5000:
                lyrics_list = [" ".join(lyrics.split()[:len(lyrics.split())//2]), " ".join(lyrics.split()[len(lyrics.split())//2:])]
            else:
                lyrics_list.append(lyrics)
        except:
            continue
        result_list = comprehend.batch_detect_entities(TextList=lyrics_list, LanguageCode='en')['ResultList']
        for result in result_list:
            entities = result['Entities']
            shoutout = list(filter(lambda entity: entity['Type'] == 'PERSON' and entity['Text'] in players, entities))
            for so in shoutout:
                if so["Text"] == "Michael Jackson":
                    continue
                shoutouts.append( (artist,so["Text"],song,mp.current_process().pid) )
#                 queue.put((artist,so["Text"],song,mp.current_process().pid))
#                 print(f'{artist} -> {so["Text"]} : {song} discovered by {mp.current_process().pid}')
    return shoutouts

In [5]:
comprehend = boto3.client('comprehend')
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
songs = []
run("Kendrick Lamar")

[]

In [6]:
players = get_basketball_players()

In [6]:
artists = get_artists_or_groups()
shoutouts = []
players = get_basketball_players()
songs = []

# cpu_count = mp.cpu_count()
# manager = mp.Manager()
# queue = manager.Queue()
# with mp.Pool( cpu_count + 2 ) as p:
#     watcher = p.map_async(poll, (queue,))
#     p.map_async(run, artists)
#     p.join()
# queue.put('-1')

KeyboardInterrupt: 

In [12]:
import pandas as pd
pd.read_csv('out.csv')

ParserError: Error tokenizing data. C error: Expected 5 fields in line 530, saw 7
